In [1]:
import torch
import torch.cuda
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F



In [2]:
transforms.Compose([transforms.ToTensor()])


In [3]:
# show train image sample 
# show train image transformed ToTensor.

In [4]:


#transform : dataset each element transform type. 
train = datasets.MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization.
]), )

test = datasets.MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization.
]), )

# Create DataLoader
dataloader_args = dict(shuffle=True, batch_size=64,num_workers=1, pin_memory=True)
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.fc1 = nn.Linear(784, 548)
        self.bc1 = self.fc1#nn.BatchNorm1d(548)

        self.fc2 = nn.Linear(548, 252)
        self.bc2 = self.fc2#nn.BatchNorm1d(252)

        self.fc3 = nn.Linear(252, 10)


    def forward(self, x):
        x = x.view((-1, 784))
        h = self.fc1(x)
        #h = self.bc1(h)
        h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)

        h = self.fc2(h)
        #h = self.bc2(h)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)

        h = self.fc3(h)
        out = F.log_softmax(h)
        return out

model = Model()
model.cuda() # CUDA!
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
model.train()

losses = []
for epoch in range(20):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        data, target = Variable(data.cuda()), Variable(target.cuda())

        # Init
        optimizer.zero_grad()

        # Predict
        y_pred = model(data)

        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.data[0])
        # Backpropagation
        loss.backward()
        optimizer.step()


        # Display
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0]),
                end='')

    print()

 Train Epoch: 0 [57664/60000 (96%)]	Loss: 0.289274
 Train Epoch: 1 [57664/60000 (96%)]	Loss: 0.156202
 Train Epoch: 2 [57664/60000 (96%)]	Loss: 0.212817
 Train Epoch: 3 [57664/60000 (96%)]	Loss: 0.030911
 Train Epoch: 4 [57664/60000 (96%)]	Loss: 0.140224
 Train Epoch: 5 [57664/60000 (96%)]	Loss: 0.032377
 Train Epoch: 6 [57664/60000 (96%)]	Loss: 0.035870
 Train Epoch: 7 [57664/60000 (96%)]	Loss: 0.020710
 Train Epoch: 8 [57664/60000 (96%)]	Loss: 0.032607
 Train Epoch: 9 [57664/60000 (96%)]	Loss: 0.011913
 Train Epoch: 10 [57664/60000 (96%)]	Loss: 0.091848
 Train Epoch: 11 [57664/60000 (96%)]	Loss: 0.102390
 Train Epoch: 12 [57664/60000 (96%)]	Loss: 0.130160
 Train Epoch: 13 [57664/60000 (96%)]	Loss: 0.005826
 Train Epoch: 14 [57664/60000 (96%)]	Loss: 0.031912
 Train Epoch: 15 [57664/60000 (96%)]	Loss: 0.004866
 Train Epoch: 16 [57664/60000 (96%)]	Loss: 0.011271
 Train Epoch: 17 [57664/60000 (96%)]	Loss: 0.003182
 Train Epoch: 18 [57664/60000 (96%)]	Loss: 0.002415
 Train Epoch: 19 [5766